This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

# Import librairies
- please note chromadb was pinned to 0.4.15 to be compatible with langchain OpenAIEmbeddings parameters (incompatibility issue https://github.com/langchain-ai/langchain/discussions/14411)

In [540]:
import os, json
from IPython.display import Markdown, display
import pandas as pd
import numpy as np
from pprint import pprint

from dotenv import load_dotenv, find_dotenv
# load up the .env file with secrets
_ = load_dotenv(find_dotenv())

from pydantic import BaseModel, Field, NonNegativeInt
from typing import List, Optional, Type

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.output_parsers import PydanticOutputParser
from langchain.document_loaders import CSVLoader  #JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import (ChatPromptTemplate,
                               PromptTemplate,
                               SystemMessagePromptTemplate,
                               AIMessagePromptTemplate,
                               HumanMessagePromptTemplate,
                               )
from langchain.schema import AIMessage, HumanMessage, SystemMessage, FunctionMessage
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools import BaseTool, format_tool_to_openai_function
from langchain import LLMChain

import openai

# Key modules and parameters

In [13]:
# colors definition
ENDC = '\33[0m'
RED = '\33[31m'
GREEN = '\33[32m'
YELLOW = '\33[33m'
VIOLET = '\33[35m'
CYAN = '\33[36m'

In [129]:
# Instantiate an OpenAI API client using langchain
instruct_llm = OpenAI(model="gpt-3.5-turbo-instruct", 
             openai_api_key=os.getenv('OPENAI_API_KEY'),
             temperature=0, 
             max_tokens=3500)

chat_llm = ChatOpenAI(model="gpt-3.5-turbo", 
             openai_api_key=os.getenv('OPENAI_API_KEY'),
             temperature=0, 
             max_tokens=3500)

import chromadb
persist_directory = './data/vectordb'
client = chromadb.PersistentClient(path=persist_directory)

embeddings_model = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))

# STEP 1 - GENERATE SYNTHETIC REAL ESTATE LISTINGS
- we want to generate synthetic listings to use as our test database
- we will use chatGPT to generate these listings

In [4]:
# Prepare property description examples for two-shot learning
listing_examples = ['''Headline: A 3-bedroom, 2-bathroom home in Green Oaks
Neighborhood: Green Oaks
Price: €1,200,000
Bedrooms: 3
Bathrooms: 2
House Size (sqm): 110

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.''',
'''Headline: Charming Oasis in Neuilly
Neighborhood: Neuilly
Price: €2,000,000
Bedrooms: 5
Bathrooms: 2
House Size (sqm): 180

Description: Discover this enchanting family home nestled in the heart of Neuilly. With its timeless elegance and modern amenities, it’s the perfect sanctuary for your family.

Neighborhood Description: Neuilly offers a serene lifestyle, top-rated schools, and picturesque parks. Enjoy the tranquility while being conveniently close to amenities.
''']
# convert examples to one string
listing_samples = '\n---------------------------------\n'.join(listing_examples)

# Pydantic models for the listings data
class listings_description_model(BaseModel):
    listings_headline: str = Field(description='property headline')
    listings_neighborhood: str = Field(description='property neighborhood where the home is located')
    listings_price: NonNegativeInt = Field(description='property price')
    listings_bedrooms: NonNegativeInt = Field(description='property number of bedrooms')
    listings_bathrooms: NonNegativeInt = Field(description='property number of bathrooms')
    listings_house_size: NonNegativeInt = Field(description='property house size')
    listings_description: str = Field(description='property description following the guidelines')
    listings_neighborhood_description: str = Field(description='property neighborhood description')

class listings_descriptions(listings_description_model):
    list_of_listings: List[listings_description_model] = Field(description='list of property descriptions following the guidelines')

parser = PydanticOutputParser(pydantic_object=listings_descriptions)
print(parser.get_format_instructions())


# Prompt to generate synthetic listing data using examples and pydantic model
template_genlistings = '''You are a real estate agent tasked with creating property listings for a variety of homes in different neighborhoods. Your goal is to craft compelling descriptions that entice potential buyers. Follow these guidelines for each listing:
1. Headline: Begin with an attention-grabbing opening sentence that captures the viewer's interest in the home.
2. In Bullet points:
    - Neighborhood: The name of the neighborhood where the home is located.
    - **Price**: Mention the price.
    - **Bedrooms**: Specify the number of bedrooms (e.g., 5).
    - **Bathrooms**: Specify the number of bathrooms (e.g., 2).
    - **House Size in sqm**: Provide the exact square footage (e.g., 150).
3. Property Description: Include a brief description of the home and its features.
4. Neighborhood description: Briefly describe the neighborhood.

========== EXEMPLES ==========
{listing_samples}
========== END OF EXAMPLES ==========

Remember to be succinct, informative, and persuasive.
**Only use the requested format.**
{format_instructions}
'''

prompt = PromptTemplate.from_template(template_genlistings,
                                      partial_variables={'format_instructions':parser.get_format_instructions})

system_prompt = prompt.format(listing_samples=listing_samples)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"listings_description_model": {"properties": {"listings_headline": {"description": "property headline", "title": "Listings Headline", "type": "string"}, "listings_neighborhood": {"description": "property neighborhood where the home is located", "title": "Listings Neighborhood", "type": "string"}, "listings_price": {"description": "property price", "minimum": 0, "title": "Listings Price", "type": "integer"}, "listings_bedrooms": {"description": "property number of bedrooms", "minimum": 0, "title": "Listings Bedrooms", "type": "intege

In [108]:
# Call GPT-3 to generate 15 listings following above guidelines.
response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
          {
            "role": "system",
            "content": system_prompt
          },
          {
            "role": "user",
            "content": "Generate 15 listings following these guidelines."
          }
          ],
        temperature=0,
        max_tokens=4000,
        )

#### RUBRIC #1 Generating Real Estate Listings with an LLM

- post processing to extract the listings

In [110]:
print(response.choices[0].message['content'][:-2])

{
    "listings_headline": "Modern Luxury in Central Park",
    "listings_neighborhood": "Central Park",
    "listings_price": 3500000,
    "listings_bedrooms": 4,
    "listings_bathrooms": 3,
    "listings_house_size": 280,
    "listings_description": "Indulge in the epitome of modern luxury with this stunning 4-bedroom, 3-bathroom home in Central Park. Boasting high-end finishes, a gourmet kitchen, and panoramic views of the city skyline, this property offers a sophisticated urban lifestyle.",
    "listings_neighborhood_description": "Central Park is a vibrant neighborhood known for its upscale dining, shopping, and cultural attractions. Enjoy easy access to the park for leisurely strolls and outdoor activities."
},
{
    "listings_headline": "Seaside Retreat in Ocean View",
    "listings_neighborhood": "Ocean View",
    "listings_price": 1800000,
    "listings_bedrooms": 3,
    "listings_bathrooms": 2,
    "listings_house_size": 200,
    "listings_description": "Escape to this charm

In [111]:
property_listings = {}
for i, desc in enumerate(eval(response.choices[0].message['content'][:-2].replace('\n', ''))):
    property_listings[i] = desc

In [138]:
for i, listing in property_listings.items():
    print(YELLOW + f"Property {str(i)} : {listing['listings_headline']}" + ENDC)
    for key in list(listings_description_model.model_fields.keys())[1:]:
        print(f'- {key.split("listings_")[1].replace("_"," "):<25} : {listing[key]}')
    print('-------------------------------------------------')

Property 0 : Modern Luxury in Central Park
- neighborhood              : Central Park
- price                     : 3500000
- bedrooms                  : 4
- bathrooms                 : 3
- house size                : 280
- description               : Indulge in the epitome of modern luxury with this stunning 4-bedroom, 3-bathroom home in Central Park. Boasting high-end finishes, a gourmet kitchen, and panoramic views of the city skyline, this property offers a sophisticated urban lifestyle.
- neighborhood description  : Central Park is a vibrant neighborhood known for its upscale dining, shopping, and cultural attractions. Enjoy easy access to the park for leisurely strolls and outdoor activities.
-------------------------------------------------
Property 1 : Seaside Retreat in Ocean View
- neighborhood              : Ocean View
- price                     : 1800000
- bedrooms                  : 3
- bathrooms                 : 2
- house size                : 200
- description         

- save the synthetic listing descriptions
- we concatenate the descriptions in one column

In [296]:
df = pd.DataFrame(property_listings).T
df['full_description'] = df.apply(lambda x: x['listings_headline'] + ': ' + x['listings_description'] + ' Neighborhood: ' + x['listings_neighborhood_description'], axis=1)
df.to_csv('data/property_listings.csv', index=False, header=True)
df.to_json('data/property_listings.json', orient='records')
df

,listings_headline,listings_neighborhood,listings_price,listings_bedrooms,listings_bathrooms,listings_house_size,listings_description,listings_neighborhood_description,full_description
0,Modern Luxury in Central Park,Central Park,3500000,4,3,280,Indulge in the epitome of modern luxury with t...,Central Park is a vibrant neighborhood known f...,Modern Luxury in Central Park: Indulge in the ...
1,Seaside Retreat in Ocean View,Ocean View,1800000,3,2,200,Escape to this charming seaside retreat in Oce...,Ocean View is a picturesque coastal community ...,Seaside Retreat in Ocean View: Escape to this ...
2,Historic Elegance in Heritage District,Heritage District,1200000,5,4,320,Step back in time with this historic 5-bedroom...,The Heritage District is a historic neighborho...,Historic Elegance in Heritage District: Step b...
3,Contemporary Living in Skyline Heights,Skyline Heights,2800000,6,5,400,Experience contemporary living at its finest i...,Skyline Heights offers a vibrant urban lifesty...,Contemporary Living in Skyline Heights: Experi...
4,Tranquil Haven in Willow Creek,Willow Creek,1500000,4,3,250,Find peace and serenity in this tranquil 4-bed...,Willow Creek is a quiet residential neighborho...,Tranquil Haven in Willow Creek: Find peace and...
5,Urban Chic in Downtown District,Downtown District,2000000,3,2,180,Embrace urban chic living in this stylish 3-be...,The Downtown District is a bustling urban cent...,Urban Chic in Downtown District: Embrace urban...
6,Rustic Charm in Maple Grove,Maple Grove,1300000,4,3,220,"Discover the rustic charm of this 4-bedroom, 3...",Maple Grove is a quaint neighborhood with tree...,Rustic Charm in Maple Grove: Discover the rust...
7,Luxurious Retreat in Hillside Manor,Hillside Manor,3000000,5,4,350,Escape to a luxurious retreat in the prestigio...,Hillside Manor is an exclusive neighborhood kn...,Luxurious Retreat in Hillside Manor: Escape to...
8,Coastal Living in Seaview Heights,Seaview Heights,2200000,3,2,190,Experience the beauty of coastal living in thi...,Seaview Heights is a coastal community with sa...,Coastal Living in Seaview Heights: Experience ...
9,Elegant Oasis in Rosewood Estates,Rosewood Estates,2600000,4,3,240,Discover an elegant oasis in the prestigious R...,Rosewood Estates is an upscale neighborhood kn...,Elegant Oasis in Rosewood Estates: Discover an...


In [519]:
#export DataFrame to text file
'''with open('./data/listings.txt', 'w') as f:
    df_string = df.to_string(header=True, index=True)
    f.write(df_string)'''

# STEP 2 - POPULATE VECTORSTORE WITH SYNTHETIC LISTINGS

In [306]:
'''loader = CSVLoader(file_path='data/property_listings.csv') #, metadata_columns=df.columns.tolist()
docs = loader.load()
print(f'Number of properties: {len(docs)}')
print(docs[0].page_content)'''
print()

PREPARE DATA FOR EMBEDDING STEP
- JSONLoader from langchain requires jq package not available on windows.
- We define a specific JSONLoader to capture the metadata of the JSON file
- code from here: https://github.com/langchain-ai/langchain/issues/4396

In [5]:
import json
from pathlib import Path
from typing import Callable, Dict, List, Optional, Union

from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader

class JSONLoader(BaseLoader):
    def __init__(
        self,
        file_path: Union[str, Path],
        content_key: Optional[str] = None,
        metadata_func: Optional[Callable[[dict, dict], dict]] = None,
    ):
        """
        Initializes the JSONLoader with a file path, an optional content key to extract specific content,
        and an optional metadata function to extract metadata from each record.
        """
        self.file_path = Path(file_path).resolve()
        self.content_key = content_key 
        self.metadata_func = metadata_func

    def create_documents(self, processed_data):
        """
        Creates Document objects from processed data.
        """
        documents = []
        for item in processed_data:
            content = item.get('content', '')  
            metadata = item.get('metadata', {})
            document = Document(page_content=content, metadata=metadata)
            documents.append(document)
        return documents

    def process_json(self, data):
        """
        Processes JSON data to prepare for document creation, extracting content based on the content_key
        and applying the metadata function if provided.
        """
        processed_data = []
        if isinstance(data, list):
            for item in data:
                content = item.get(self.content_key, '') if self.content_key else ''
                metadata = {}
                if self.metadata_func and isinstance(item, dict):
                    metadata = self.metadata_func(item, {})
                processed_data.append({'content': content, 'metadata': metadata})
        return processed_data

    def load(self) -> List[Document]:
        """
        Load and return documents from the JSON file.
        """
        docs = []
        with open(self.file_path, mode="r", encoding="utf-8") as json_file:
            try:
                data = json.load(json_file)
                processed_json = self.process_json(data)
                docs = self.create_documents(processed_json)
            except json.JSONDecodeError:
                print("Error: Invalid JSON format in the file.")
        return docs

- we will use the combined description for the semantic search
- we will use price, # rooms, house size and neighborhood in the metadata for easier, more accurate 'keyword' search

In [6]:
def metadata_func(item, _):
    return {
            'neighborhood':item['listings_neighborhood'],
            'price': item['listings_price'],
            'bedrooms':item['listings_bedrooms'],
            'bathrooms':item['listings_bathrooms'],
            'house size':item['listings_house_size'],
            }

json_loader= JSONLoader('data/property_listings.json',content_key='full_description', metadata_func=metadata_func)
docs = json_loader.load()
docs

[Document(page_content='Modern Luxury in Central Park: Indulge in the epitome of modern luxury with this stunning 4-bedroom, 3-bathroom home in Central Park. Boasting high-end finishes, a gourmet kitchen, and panoramic views of the city skyline, this property offers a sophisticated urban lifestyle. Neighborhood: Central Park is a vibrant neighborhood known for its upscale dining, shopping, and cultural attractions. Enjoy easy access to the park for leisurely strolls and outdoor activities.', metadata={'neighborhood': 'Central Park', 'price': 3500000, 'bedrooms': 4, 'bathrooms': 3, 'house size': 280}),
 Document(page_content='Seaside Retreat in Ocean View: Escape to this charming seaside retreat in Ocean View. With 3 bedrooms, 2 bathrooms, and a spacious deck overlooking the ocean, this home offers a perfect blend of coastal living and modern comfort. Neighborhood: Ocean View is a picturesque coastal community with sandy beaches, seafood restaurants, and a laid-back atmosphere. Experien

- For demonstration purpose we implement a text splitter. However to ensure semantic search we embed the full description as a whole since it is of limited size anyhow

In [9]:
'''# Splitter to chunk text into smaller chunks
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

split_docs = text_splitter.split_documents(docs)

# embed each chunk and load it into the vector store.
db = Chroma.from_documents(split_docs, OpenAIEmbeddings())'''
print()

### RUBRIC #2 Creating a Vector Database and Storing Listings

In [16]:
# we create a chromadb collection to store the listings, their embeddings and metadata
try:
    collection = client.get_collection(name="homematch", embedding_function=OpenAIEmbeddings())
except:
    collection = client.create_collection(name="homematch", embedding_function=OpenAIEmbeddings())

In [21]:
# we generate the ambeddings for each description using OpenAi Embedding model
embeddings = embeddings_model.embed_documents([doc.page_content for doc in docs])

In [30]:
# we add the data to the collection - this is automatically persisted to disk in the data/db folder
collection.add(
    documents=[doc.page_content for doc in docs],
    embeddings=embeddings,
    metadatas=[doc.metadata for doc in docs],
    ids=[str(i) for i in range(len(docs))] 
)

In [84]:
collection.count() # returns the number of items in the collection

16

In [34]:
collection.peek(2) # returns a list of the first 2 items in the collection

{'ids': ['0', '1'],
 'embeddings': [[0.012835898850191383,
   0.023745753775894586,
   -0.009729163328777135,
   -0.013508695564182034,
   0.0022014292466056174,
   0.018007195759674403,
   -0.014379373171587419,
   0.0029847092188278394,
   -0.03350789709835375,
   -0.0130139919336614,
   -0.0005907580833086552,
   0.01776973831504771,
   -0.022255047258125295,
   -0.005339496553641952,
   -0.0010479462375994788,
   0.0002786828150790133,
   0.0322414549101513,
   -0.017505896502946375,
   0.011490306353532613,
   -0.02121287312477982,
   -0.036805918818297874,
   0.0114045576249013,
   0.006371777247676338,
   -0.011648611627057921,
   -0.004933839930517597,
   -0.0013662053419559895,
   0.010843894007016602,
   -0.008066960285067752,
   0.011364982005011875,
   0.0029962524960133116,
   0.004854687293754943,
   -0.0063289028833606814,
   -0.027360386042889324,
   -0.005398860914798625,
   -0.00310014012803749,
   -0.017756547062632928,
   -0.004171997140453203,
   -0.020988608795212

# STEP 3 - SEMANTIC SEARCH FONCTIONALITY

In [254]:
# load property database to pandas
df = pd.read_json('data/property_listings.json')
df

,listings_headline,listings_neighborhood,listings_price,listings_bedrooms,listings_bathrooms,listings_house_size,listings_description,listings_neighborhood_description,full_description
0,Modern Luxury in Central Park,Central Park,3500000,4,3,280,Indulge in the epitome of modern luxury with t...,Central Park is a vibrant neighborhood known f...,Modern Luxury in Central Park: Indulge in the ...
1,Seaside Retreat in Ocean View,Ocean View,1800000,3,2,200,Escape to this charming seaside retreat in Oce...,Ocean View is a picturesque coastal community ...,Seaside Retreat in Ocean View: Escape to this ...
2,Historic Elegance in Heritage District,Heritage District,1200000,5,4,320,Step back in time with this historic 5-bedroom...,The Heritage District is a historic neighborho...,Historic Elegance in Heritage District: Step b...
3,Contemporary Living in Skyline Heights,Skyline Heights,2800000,6,5,400,Experience contemporary living at its finest i...,Skyline Heights offers a vibrant urban lifesty...,Contemporary Living in Skyline Heights: Experi...
4,Tranquil Haven in Willow Creek,Willow Creek,1500000,4,3,250,Find peace and serenity in this tranquil 4-bed...,Willow Creek is a quiet residential neighborho...,Tranquil Haven in Willow Creek: Find peace and...
5,Urban Chic in Downtown District,Downtown District,2000000,3,2,180,Embrace urban chic living in this stylish 3-be...,The Downtown District is a bustling urban cent...,Urban Chic in Downtown District: Embrace urban...
6,Rustic Charm in Maple Grove,Maple Grove,1300000,4,3,220,"Discover the rustic charm of this 4-bedroom, 3...",Maple Grove is a quaint neighborhood with tree...,Rustic Charm in Maple Grove: Discover the rust...
7,Luxurious Retreat in Hillside Manor,Hillside Manor,3000000,5,4,350,Escape to a luxurious retreat in the prestigio...,Hillside Manor is an exclusive neighborhood kn...,Luxurious Retreat in Hillside Manor: Escape to...
8,Coastal Living in Seaview Heights,Seaview Heights,2200000,3,2,190,Experience the beauty of coastal living in thi...,Seaview Heights is a coastal community with sa...,Coastal Living in Seaview Heights: Experience ...
9,Elegant Oasis in Rosewood Estates,Rosewood Estates,2600000,4,3,240,Discover an elegant oasis in the prestigious R...,Rosewood Estates is an upscale neighborhood kn...,Elegant Oasis in Rosewood Estates: Discover an...


### RUBRIC #3 Semantic Search of Listings Based on Buyer Preferences

In [81]:
def query_db(query: str,
             metadata: Optional[dict] = None,
             search_string: Optional[str] = ' ',
             n_results: int = 10
             ):
    """
    text query the database and return the n closest vector entries
    inputs:
    - query: the query string
    - metadata: the metadata to filter the results
        - Provide metadata as a dictionary with metadata keys and values as lists of values to filter for
        - targets can be composed with the following operators:
        e.g {'house size': {"$gte": 200}
            $eq - equal to (string, int, float)
            $ne - not equal to (string, int, float)
            $gt - greater than (int, float)
            $gte - greater than or equal to (int, float)
            $lt - less than (int, float)
            $lte - less than or equal to (int, float)
    - search_string: the string or keyword to search in the listing description
    - n_results: the number of results to return (int - default = 10)
    returns:
    - the n closest results as a list of dictionaries.
    The results are ordered by the distance to the query in ascending order (lowest in best match)
    Returns the ids, description, metadata and distance of the corresponding vector entries
    """
    # compute the embeddings of the query for semantic search
    query_embeddings = embeddings_model.embed_documents([query])
    # defines the keywords for the search function
    kwargs  = {'query_embeddings':query_embeddings, 'n_results':n_results}
    if metadata:
        kwargs['where'] = metadata
    if search_string:
        kwargs['where_document'] = {"$contains":search_string}
    # serach the vector db using the defined arguments & return results
    results = collection.query(**kwargs)

    return results

In [79]:
query = "nice house with a view on the sea"
query_db(query=query, n_results=3)

{'ids': [['8', '1', '11']],
 'distances': [[0.2822312941790316, 0.2836603463282615, 0.4023989250458718]],
 'metadatas': [[{'bathrooms': 2,
    'bedrooms': 3,
    'house size': 190,
    'neighborhood': 'Seaview Heights',
    'price': 2200000},
   {'bathrooms': 2,
    'bedrooms': 3,
    'house size': 200,
    'neighborhood': 'Ocean View',
    'price': 1800000},
   {'bathrooms': 2,
    'bedrooms': 3,
    'house size': 210,
    'neighborhood': 'Silver Lake',
    'price': 1900000}]],
 'embeddings': None,
 'documents': [['Coastal Living in Seaview Heights: Experience the beauty of coastal living in this 3-bedroom, 2-bathroom home in Seaview Heights. With ocean views, a sun-drenched deck, and beach access just steps away, this property offers a perfect seaside retreat. Neighborhood: Seaview Heights is a coastal community with sandy beaches, seaside cafes, and water sports activities. Enjoy the laid-back lifestyle and stunning ocean views in this charming neighborhood.',
   'Seaside Retreat in

In [68]:
query_db(query=query, n_results=3, metadata={'house size': {"$gte": 200}})

{'ids': [['1', '11', '3']],
 'distances': [[0.2836603463282615, 0.4023989250458718, 0.40659283973841437]],
 'metadatas': [[{'bathrooms': 2,
    'bedrooms': 3,
    'house size': 200,
    'neighborhood': 'Ocean View',
    'price': 1800000},
   {'bathrooms': 2,
    'bedrooms': 3,
    'house size': 210,
    'neighborhood': 'Silver Lake',
    'price': 1900000},
   {'bathrooms': 5,
    'bedrooms': 6,
    'house size': 400,
    'neighborhood': 'Skyline Heights',
    'price': 2800000}]],
 'embeddings': None,
 'documents': [['Seaside Retreat in Ocean View: Escape to this charming seaside retreat in Ocean View. With 3 bedrooms, 2 bathrooms, and a spacious deck overlooking the ocean, this home offers a perfect blend of coastal living and modern comfort. Neighborhood: Ocean View is a picturesque coastal community with sandy beaches, seafood restaurants, and a laid-back atmosphere. Experience the tranquility of seaside living in this charming neighborhood.',
   'Modern Elegance in Silver Lake: Expe

In [69]:
query_db(query=query, n_results=3, metadata={'house size': {"$gte": 200}}, search_string='sea')

{'ids': [['1']],
 'distances': [[0.2836603463282615]],
 'metadatas': [[{'bathrooms': 2,
    'bedrooms': 3,
    'house size': 200,
    'neighborhood': 'Ocean View',
    'price': 1800000}]],
 'embeddings': None,
 'documents': [['Seaside Retreat in Ocean View: Escape to this charming seaside retreat in Ocean View. With 3 bedrooms, 2 bathrooms, and a spacious deck overlooking the ocean, this home offers a perfect blend of coastal living and modern comfort. Neighborhood: Ocean View is a picturesque coastal community with sandy beaches, seafood restaurants, and a laid-back atmosphere. Experience the tranquility of seaside living in this charming neighborhood.']]}

# STEP 4 - BUILD USER PREFERENCE INTERFACE

- prepare schema and output parser to extract key preference characteristics from customer message

In [370]:
# Keep this cell formating as it influences the llm output with int instead of 'int' !
price_schema = ResponseSchema(name="price",
                              description="Did the customer indicated a maximum price for the property? \
                                Provide the price value if yes, -1 if not or unknown.")
bathrooms_schema = ResponseSchema(name="bathrooms",
                                  description="Did the customer indicated the number of bathrooms for the property? \
                                    Provide the number value if yes, -1 if not or unknown.")
bedrooms_schema = ResponseSchema(name="bedrooms",
                                 description="Did the customer indicated the number of bedrooms for the property? \
                                    Provide the number value if yes, -1 if not or unknown.")
size_schema = ResponseSchema(name="house size",
                             description="Did the customer indicated the minimum size of the property? \
                                Provide the number value if yes, -1 if not or unknown.")

response_schemas = [price_schema, 
                    bathrooms_schema,
                    bedrooms_schema,
                    size_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_user_preferences = output_parser.get_format_instructions()
print(format_user_preferences)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"price": string  // Did the customer indicated a maximum price for the property?                                 Provide the price value if yes, -1 if not or unknown.
	"bathrooms": string  // Did the customer indicated the number of bathrooms for the property?                                     Provide the number value if yes, -1 if not or unknown.
	"bedrooms": string  // Did the customer indicated the number of bedrooms for the property?                                     Provide the number value if yes, -1 if not or unknown.
	"house size": string  // Did the customer indicated the minimum size of the property?                                 Provide the number value if yes, -1 if not or unknown.
}
```


In [371]:
def build_where_expression(dict_of_metadata):
    """
    Builds a where_document for metadata search, format expected by vectorDB
    """
    # eliminate all metadata keys with no value identified (-1)
    dict_of_metadata = {k:v for k,v in dict_of_metadata.items() if v != -1}
    # build metadata search expression using Where format expected by vectorDB
    expression = {}
    try:
        max_price = dict_of_metadata['price']
        # use less than or equal operator for price
        dict_of_metadata['price'] = {"$lte": max_price}
    except:
        pass
    if len(dict_of_metadata) >1:
        # combine all metadata search expressions into a single Where expression
        expression = {"$and": [{k:v} for k,v in dict_of_metadata.items()]}
    return expression

In [372]:
# Prompt template for finding user preferences.
# It uses the schema defined above
find_preferences_template = """\
For the following customer input, extract the following information:

price: Did the customer indicated a maximum price for the property? \
Provide the price value if yes, -1 if not or unknown.

bathrooms: Did the customer indicated the number of bathrooms for the property? \
Provide the number value if yes, -1 if not or unknown.

bedrooms: Did the customer indicated the number of bedrooms for the property? \
Provide the number value if yes, -1 if not or unknown.

House size: Did the customer indicated the minimum size of the property? \
Provide the number value if yes, -1 if not or unknown.

customer input: {customer_input}

{format_user_preferences}
"""

In [373]:
def find_best_matches(customer_input, top_k=3):
    '''   
    takes in the customer text input indicating his preferences, find the best property matches and extract customer preference characteristics as metadata
    returns :
    - the top k (default = 3) best property matches using query_db function
    - the key characteristics indicated by the customer for the preperty metadata

    input:
    - customer_input: str
    outputs:
    - listings_results: dictionary with property matches from query_db function
    - output_dict: dictionary with key, value pairs representing the metadata and their extracted values
    '''
    pref_prompt = ChatPromptTemplate.from_template(template=find_preferences_template)
    messages = pref_prompt.format_messages(customer_input=customer_input, 
                                  format_user_preferences=format_user_preferences)

    # Use LLM to extratc the preferences from the conversation
    response = chat_llm(messages)
    output_dict = output_parser.parse(response.content)
    print(f'Extracted desired preferencies: {output_dict}')

    # Query the vectordb with the customer input and the key preference characteristics on metadata
    listings_results = query_db(query=customer_input, 
                                n_results=top_k, 
                                metadata=build_where_expression(output_dict)
                                )
    return listings_results, output_dict

In [377]:
def display_matches(matches, threshold=0.52, df=df):
    '''   
    takes in the list of property matches from query_db function
    displays the property characterics from the df dataframe
    filter out the matches with distance above a threshold (default is 0.52)
    inputs:
    - property matches from query_db function
    - threshold: float between 0 and 1. lower is more restrictive
    - df: dataframe with the properties details (default is database)
    outputs:
    - matches_df: input dataframe filtered to property matches below threshold
    '''
    idx = matches['ids'][0]
    idx = [int(i) for i in idx]
    if len(idx)>0:
        idx = [id for i,id in enumerate(idx) if matches['distances'][0][i]<=threshold]
        
    if len(idx)>0:
        matches_df = df.loc[idx]
        
        for i, id in enumerate(idx):
                print(YELLOW + f"Distance {str(round(matches['distances'][0][i],3))} : {matches_df.loc[id,'listings_headline']}" + ENDC)
                for key in list(listings_description_model.model_fields.keys())[1:]:
                    print(f'- {key.split("listings_")[1].replace("_"," "):<25} : {matches_df.loc[id,key]}')
                print('-------------------------------------------------')
    else:
        matches_df = pd.DataFrame()
        print(RED + "No match found with these characteristics" + ENDC)

    return matches_df

### RUBRIC #3 Semantic Search of Listings Based on Buyer Preferences

In [378]:
customer_input = "i am looking for a property with a view on the sea with 3 bedrooms, 3 bathrooms and maximum price of 2000000 euros"

extrated_properties, extracted_prefs = find_best_matches(customer_input)

matches_df = display_matches(extrated_properties)

Extracted desired preferencies: {'price': 2000000, 'bathrooms': 3, 'bedrooms': 3, 'house size': -1}
No match found with these characteristics


In [379]:
customer_input = "i am looking for a property with a view on the sea with 3 bedrooms, 2 bathrooms and maximum price of 2000000 euros"

extrated_properties, extracted_prefs = find_best_matches(customer_input)

matches_df = display_matches(extrated_properties)

Extracted desired preferencies: {'price': 2000000, 'bathrooms': 2, 'bedrooms': 3, 'house size': -1}
Distance 0.439 : Seaside Retreat in Ocean View
- neighborhood              : Ocean View
- price                     : 1800000
- bedrooms                  : 3
- bathrooms                 : 2
- house size                : 200
- description               : Escape to this charming seaside retreat in Ocean View. With 3 bedrooms, 2 bathrooms, and a spacious deck overlooking the ocean, this home offers a perfect blend of coastal living and modern comfort.
- neighborhood description  : Ocean View is a picturesque coastal community with sandy beaches, seafood restaurants, and a laid-back atmosphere. Experience the tranquility of seaside living in this charming neighborhood.
-------------------------------------------------
Distance 0.51 : Modern Elegance in Silver Lake
- neighborhood              : Silver Lake
- price                     : 1900000
- bedrooms                  : 3
- bathrooms      

# STEP 5 - AUGMENT AND PERSONALIZE LISTING DESCRIPTION TO CUSTOMER PREFERENCES

### RUBRIC #4 Logic for Searching and Augmenting Listing Descriptions

- summarize the preferences of the customer

In [301]:
summarize_preferences_prompt_template = ''' 
Based on the customer input and extracted perference characterisctics below, summarize the user preferencies describing the ideal property the customer is looking for.
- Be concise.
- Only use what is provided to you. Do not alter or modify preferencies.
- Do not add preferencies not mentionned by the customer as this could flaw the conclusion.
- if you do not know, say "I do not know"
customer input: {customer_input}
extracted preferencies: {extracted_preferences}
'''
summ_prompt = PromptTemplate.from_template(summarize_preferences_prompt_template)

In [302]:
summarizing_prompt = summ_prompt.format(customer_input=query, extracted_preferences = extracted_prefs)

In [303]:
summarized_preferences = instruct_llm(summarizing_prompt)
print(summarized_preferences)


The customer is looking for a 3 bedroom, 2 bathroom house with a view of the sea, with a budget of $2,000,000.


- Use to augment descriptions and personalize to the expectations 

In [380]:
augmentation_prompt_template = '''
You are a real estate agent with great style to emphasize the property features to the customer preferences.
Based on the description of what the customer is looking for, the property description and property characteristics, rewrite a personalized, catchy, brief on the property enhancing those features.
- Be concise.
- Stick to what is provided to you. Do not alter or modify the customer preferences.
- only emphasize confirmed characteristics
- Do not invent preferences not mentionned by the customer since this could negatively flaw the conclusion.
- if you do not know, say "I do not know"
property descriptiont: {property_description}
customer preferences: {summarized_preferences}
'''
aug_prompt = PromptTemplate.from_template(augmentation_prompt_template)

In [403]:
def augment_properties(properties,summarized_preferences, df):
    '''   
    Takes in best property matches, summary of buyer preferences and initial property description
    Generates an augmented description for each selected property enhancing the buyer's features
    Display the augmented descriptions together with the exact characteristics of the properties
    inputs:
    - best property matches from find_best_matches function
    - summarized buyer preferences (str)
    - dataframe with detailed characteristics of the bast matches
    outputs:
    - dataframe with detailed characteristics of the bast matches and personalized, augmented, description using an LLM
    '''
    idx = df.index.to_list()
    
    keys = ['listings_neighborhood',
        'listings_price',
        'listings_bedrooms',
        'listings_bathrooms',
        'listings_house_size',
        #'listings_augmented_description',
        ]
    
    if len(idx) > 0:
        for id in idx:
            full_description = df.loc[id, 'full_description']
            augmentation_prompt = aug_prompt.format(property_description=full_description, summarized_preferences = summarized_preferences)
            augmented_desc = instruct_llm(augmentation_prompt)
            df.loc[id, 'augmented_description'] = augmented_desc
        
        for i, id in enumerate(idx):
            print(YELLOW + f"Distance {str(round(properties['distances'][0][i],3))} : {df.loc[id,'listings_headline']}" + ENDC)
            for key in keys:
                print(f'- {key.split("listings_")[1].replace("_"," "):<25} : {df.loc[id,key]}')
            
            display(Markdown('<span style="color:cyan">'+f'augmented description : {df.loc[id,"augmented_description"]}'+'</span>'))
            print('-------------------------------------------------')

    else:
        print(RED + "No match found with these characteristics" + ENDC)

    return df

### RUBRIC #5 Use of LLM for Generating Personalized Descriptions

In [404]:
augmented_df = augment_properties(extrated_properties, summarized_preferences, matches_df)

Distance 0.439 : Seaside Retreat in Ocean View
- neighborhood              : Ocean View
- price                     : 1800000
- bedrooms                  : 3
- bathrooms                 : 2
- house size                : 200


<span style="color:cyan">augmented description : 
Welcome to your dream seaside retreat in Ocean View! This charming home boasts 3 bedrooms and 2 bathrooms, perfect for your family's needs. Step out onto the spacious deck and take in the breathtaking view of the ocean. Imagine waking up to the sound of waves crashing and enjoying your morning coffee with a stunning view. With a budget of $2,000,000, this home offers the perfect blend of coastal living and modern comfort. Located in the picturesque neighborhood of Ocean View, you'll have access to sandy beaches, delicious seafood restaurants, and a laid-back atmosphere. Don't miss out on the opportunity to experience the tranquility of seaside living in this charming community. </span>

-------------------------------------------------
Distance 0.51 : Modern Elegance in Silver Lake
- neighborhood              : Silver Lake
- price                     : 1900000
- bedrooms                  : 3
- bathrooms                 : 2
- house size                : 210


<span style="color:cyan">augmented description : 
"Experience modern elegance in this stunning 3-bedroom, 2-bathroom home located in the trendy Silver Lake neighborhood. With sleek design and high ceilings, this property exudes sophistication. But the real showstopper? The rooftop deck with panoramic views of the lake, perfect for enjoying the vibrant energy and creative spirit of this eclectic community. And with a budget of $2,000,000, this is the perfect opportunity to own a piece of Silver Lake's hip cafes, art galleries, and scenic lake views. Don't miss out on this modern gem!"</span>

-------------------------------------------------


In [407]:
augmented_df

,listings_headline,listings_neighborhood,listings_price,listings_bedrooms,listings_bathrooms,listings_house_size,listings_description,listings_neighborhood_description,full_description,augmented_description
1,Seaside Retreat in Ocean View,Ocean View,1800000,3,2,200,Escape to this charming seaside retreat in Oce...,Ocean View is a picturesque coastal community ...,Seaside Retreat in Ocean View: Escape to this ...,\nWelcome to your dream seaside retreat in Oce...
11,Modern Elegance in Silver Lake,Silver Lake,1900000,3,2,210,"Experience modern elegance in this 3-bedroom, ...",Silver Lake is a trendy neighborhood known for...,Modern Elegance in Silver Lake: Experience mod...,"\n""Experience modern elegance in this stunning..."


# EXTRA: WORKFLOW WITH TOOL + FUNCTION CALLING TO EXTRACT AND REPOND WITH PERSONALIZED DESCRIPTIONS

- define tool to search for best match in property DB using pydantic structure

In [682]:
class GetBestPropertyMatchToolInput(BaseModel):
    """Call this function to find the properties best matching customer preferences"""
    query: str = Field(description="summary of the property's characteristics the customer looks for")
    metadata: Optional[dict] = Field(description="dictionary of metadata and value pairs of the target properties such as price, number of bedrooms, number of bathrooms, house surface, neighborhood")
    search_string: Optional[str] = Field(description="keyword to search for in the property description")
    n_results: Optional[NonNegativeInt] = Field(description="the number of property match to return - default = 3")


class GetBestPropertyMatchTool(BaseTool):
    name = "query_db"
    description = "Call this function to find the properties best matching customer preferences"

    def _run(self, query:str, metadata: Optional[dict] = None, search_string: Optional[str] = None, n_results: Optional[NonNegativeInt] = 3):
        """
        Run the tool and return the best match
        """
        # Query the database
        best_matches = query_db(query, metadata, search_string, n_results)  #TODO build_where_expression(metadata) ??
        return best_matches
    
    def _arun(self, query:str, metadata: Optional[dict] = None, search_string: Optional[str] = None, n_results: Optional[NonNegativeInt] = 3):
        """
        Run the tool and return the best match
        """
        raise NotImplementedError('This tool does not support async')
    
    args_schema: Type[BaseModel] = GetBestPropertyMatchToolInput


# Declare tools and functions
tools = [GetBestPropertyMatchTool()]
functions = [format_tool_to_openai_function(tool_name) for tool_name in tools]

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=GetBestPropertyMatchToolInput)

In [683]:
tools[0]

GetBestPropertyMatchTool(name='query_db', description='Call this function to find the properties best matching customer preferences', args_schema=<class '__main__.GetBestPropertyMatchToolInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False)

In [475]:
functions

[{'name': 'query_db',
  'description': 'Call this function to find the properties best matching customer preferences',
  'parameters': {'description': 'Call this function to find the properties best matching customer preferences',
   'properties': {'query': {'description': "summary of the property's characteristics the customer looks for",
     'title': 'Query',
     'type': 'string'},
    'metadata': {'anyOf': [{'type': 'object'}, {'type': 'null'}],
     'description': 'dictionary of metadata and value pairs of the target properties such as price, number of bedrooms, number of bathrooms, house surface, neighborhood',
     'title': 'Metadata'},
    'search_string': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'description': 'keyword to search for in the property description',
     'title': 'Search String'},
    'n_results': {'anyOf': [{'minimum': 0, 'type': 'integer'},
      {'type': 'null'}],
     'description': 'the number of property match to return - default = 3',
     't

- generate function calling based on customer preferences

In [498]:
customer_input = "i am looking for a house with 3 bedrooms and a view on the lake?"
ai_response = chat_llm.predict_messages([HumanMessage(content=customer_input)], functions=functions)
# similar to response = chat_llm.invoke("i am looking for a house with 3 bedrooms and a view on the lake?", functions=functions)
ai_response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'query_db', 'arguments': '{"query":"3 bedrooms, lake view","metadata":null,"search_string":null,"n_results":3}'}})

- extract function calling output

In [514]:
if ai_response.content=='':
    tool_name = ai_response.additional_kwargs['function_call']['name']
    tool = [tools[i] for i, tool in enumerate(tools) if tool.name==tool_name][0]
    _args = json.loads(ai_response.additional_kwargs['function_call']['arguments'])
print(tool.name)
print(_args)
extrated_properties = tool(_args)
extrated_properties

query_db
{'query': '3 bedrooms, lake view', 'metadata': None, 'search_string': None, 'n_results': 3}


{'ids': [['11', '1', '6']],
 'distances': [[0.3404558433476212, 0.3557057768479667, 0.3824061701253278]],
 'metadatas': [[{'bathrooms': 2,
    'bedrooms': 3,
    'house size': 210,
    'neighborhood': 'Silver Lake',
    'price': 1900000},
   {'bathrooms': 2,
    'bedrooms': 3,
    'house size': 200,
    'neighborhood': 'Ocean View',
    'price': 1800000},
   {'bathrooms': 3,
    'bedrooms': 4,
    'house size': 220,
    'neighborhood': 'Maple Grove',
    'price': 1300000}]],
 'embeddings': None,
 'documents': [['Modern Elegance in Silver Lake: Experience modern elegance in this 3-bedroom, 2-bathroom home in Silver Lake. With sleek design, high ceilings, and a rooftop deck with panoramic views of the lake, this property offers a perfect blend of style and sophistication. Neighborhood: Silver Lake is a trendy neighborhood known for its hip cafes, art galleries, and scenic lake views. Embrace the vibrant energy and creative spirit of this eclectic community.',
   'Seaside Retreat in Ocean

- narrow down the list using threshold parameter

In [515]:
matches_df = display_matches(extrated_properties,0.37)
matches_df

Distance 0.34 : Modern Elegance in Silver Lake
- neighborhood              : Silver Lake
- price                     : 1900000
- bedrooms                  : 3
- bathrooms                 : 2
- house size                : 210
- description               : Experience modern elegance in this 3-bedroom, 2-bathroom home in Silver Lake. With sleek design, high ceilings, and a rooftop deck with panoramic views of the lake, this property offers a perfect blend of style and sophistication.
- neighborhood description  : Silver Lake is a trendy neighborhood known for its hip cafes, art galleries, and scenic lake views. Embrace the vibrant energy and creative spirit of this eclectic community.
-------------------------------------------------
Distance 0.356 : Seaside Retreat in Ocean View
- neighborhood              : Ocean View
- price                     : 1800000
- bedrooms                  : 3
- bathrooms                 : 2
- house size                : 200
- description               : Escap

,listings_headline,listings_neighborhood,listings_price,listings_bedrooms,listings_bathrooms,listings_house_size,listings_description,listings_neighborhood_description,full_description
11,Modern Elegance in Silver Lake,Silver Lake,1900000,3,2,210,"Experience modern elegance in this 3-bedroom, ...",Silver Lake is a trendy neighborhood known for...,Modern Elegance in Silver Lake: Experience mod...
1,Seaside Retreat in Ocean View,Ocean View,1800000,3,2,200,Escape to this charming seaside retreat in Oce...,Ocean View is a picturesque coastal community ...,Seaside Retreat in Ocean View: Escape to this ...


- generate augmented personalized descriptions

In [516]:
augmented_df = augment_properties(extrated_properties, customer_input, matches_df)

Distance 0.34 : Modern Elegance in Silver Lake
- neighborhood              : Silver Lake
- price                     : 1900000
- bedrooms                  : 3
- bathrooms                 : 2
- house size                : 210


<span style="color:cyan">augmented description : 
"Welcome to your dream home in Silver Lake! This stunning 3-bedroom, 2-bathroom property boasts modern elegance with its sleek design, high ceilings, and a rooftop deck offering breathtaking views of the lake. Immerse yourself in the trendy and vibrant neighborhood, known for its hip cafes, art galleries, and scenic lake views. Don't miss the opportunity to experience the perfect blend of style and sophistication in this sought-after community. Come see it for yourself!"</span>

-------------------------------------------------
Distance 0.356 : Seaside Retreat in Ocean View
- neighborhood              : Ocean View
- price                     : 1800000
- bedrooms                  : 3
- bathrooms                 : 2
- house size                : 200


<span style="color:cyan">augmented description : 
Welcome to your dream seaside retreat in Ocean View! This charming home boasts 3 bedrooms and a stunning view of the ocean from its spacious deck. Immerse yourself in the perfect blend of coastal living and modern comfort. Located in the picturesque neighborhood of Ocean View, you'll have access to sandy beaches, delicious seafood restaurants, and a laid-back atmosphere. Don't miss out on the opportunity to experience the tranquility of seaside living in this charming community. Come see it for yourself today!</span>

-------------------------------------------------


- Generate final LLM response in natural language

In [518]:
final_output = chat_llm.predict_messages([ 
    HumanMessage(content=customer_input),
    ai_response,
    FunctionMessage(name=tool.name, content=str(extrated_properties)),
    AIMessage(content=str((augmented_df
                           .drop(['listings_description','listings_neighborhood_description','full_description'], axis=1)  # we keep the augmented description and the metadata characteristics
                           .to_dict(orient='dict')
                           )
                           ))
    ], 
    functions=functions
)
display(Markdown(final_output.content))

I found some properties that match your preferences:

1. **Modern Elegance in Silver Lake**
   - **Neighborhood:** Silver Lake
   - **Price:** $1,900,000
   - **Bedrooms:** 3
   - **Bathrooms:** 2
   - **House Size:** 210 sqft
   - **Description:** Welcome to your dream home in Silver Lake! This stunning 3-bedroom, 2-bathroom property boasts modern elegance with its sleek design, high ceilings, and a rooftop deck offering breathtaking views of the lake. Immerse yourself in the trendy and vibrant neighborhood, known for its hip cafes, art galleries, and scenic lake views. Don't miss the opportunity to experience the perfect blend of style and sophistication in this sought-after community. Come see it for yourself!

2. **Seaside Retreat in Ocean View**
   - **Neighborhood:** Ocean View
   - **Price:** $1,800,000
   - **Bedrooms:** 3
   - **Bathrooms:** 2
   - **House Size:** 200 sqft
   - **Description:** Welcome to your dream seaside retreat in Ocean View! This charming home boasts 3 bedrooms and a stunning view of the ocean from its spacious deck. Immerse yourself in the perfect blend of coastal living and modern comfort. Located in the picturesque neighborhood of Ocean View, you'll have access to sandy beaches, delicious seafood restaurants, and a laid-back atmosphere. Don't miss out on the opportunity to experience the tranquility of seaside living in this charming community. Come see it for yourself today!

# FULL AUTOMATIZED WORKFLOW with automated preference collection using tagging

In [549]:
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from enum import Enum
from pydantic.v1 import BaseModel, Field  # required due to bug issue in later pydantic version

In [664]:
chat_llm = ChatOpenAI(model='gpt-3.5-turbo-1106', temperature=0.1, max_tokens=3500)

In [684]:
class PreferenceDetails(BaseModel):
    price: NonNegativeInt = Field(description="This is the maximum price the customer can afford to buy a property.",default=0)
    #planned_price: NonNegativeInt = Field(description="This is the price amount the customer would prefer to spend to buy a property.",default=0)
    #neighborhood: str = Field(description="This is the neighborhood where the customer wants to buy a property",default='')
    bedrooms: NonNegativeInt = Field(description="This is the number of desired bedrooms for the property",default=0)
    bathrooms: NonNegativeInt = Field(description="This is the number of desired bathrooms for the property",default=0)
    size: NonNegativeInt = Field(#enum=["spanish", "english", "french", "german", "italian"]
                                description="This is the minimum desired size of the property in square meters",default=0)

In [685]:
# test
tt = PreferenceDetails()
tt

PreferenceDetails(price=0, bedrooms=0, bathrooms=0, size=0)

In [686]:
chain = create_tagging_chain_pydantic(PreferenceDetails, chat_llm)

In [687]:
# test
customer_input = "i am looking for a property with a view on the sea with 3 bedrooms, 3 bathrooms and maximum price of 2000000 euros"
res = chain.run(customer_input)
res

PreferenceDetails(price=2000000, bedrooms=3, bathrooms=3, size=0)

In [688]:
# check which metadata field are still unknown and empty
def check_what_is_empty(user_preferences):
    ask_for = []
    # Check if fields are empty
    for field, value in user_preferences.dict().items():
        if value in [None, "", 0]:  # You can add other 'empty' conditions as per your requirements
            print(RED + f"Field '{field}' is empty." + ENDC)
            ask_for.append(f'{field}')
    return ask_for

## checking the response and adding it
def add_non_empty_details(current_details: PreferenceDetails, new_details: PreferenceDetails):
    non_empty_details = {k: v for k, v in new_details.dict().items() if v not in [None, 0, ""]}
    updated_details = current_details.copy(update=non_empty_details)
    return updated_details


def ask_for_info(ask_for = ['maximum price','number of bedrooms', 'neighborhood']):

    # prompt template 1
    first_prompt = ChatPromptTemplate.from_template(
        "You are an real estate agent. Below is are some things to ask the customer for in a conversation way to understand his preferences. you should only ask one question at a time even if you don't get all the info \
        don't ask as a list! Don't greet the user! Don't say Hi. Explain you need to get some info to identify the best property match.\n\n \
        ### ask_for list: {ask_for}"  #If the ask_for list is empty then thank them and ask how you can help them 
    )

    # info_gathering_chain
    info_gathering_chain = LLMChain(llm=chat_llm, prompt=first_prompt)
    ai_chat = info_gathering_chain.run(ask_for=ask_for)
    return ai_chat


def filter_response(text_input, user_preferences ):
    tagging_chain = create_tagging_chain_pydantic(PreferenceDetails, chat_llm)
    res = tagging_chain.run(text_input)
    # add filtered info to the
    user_preferences = add_non_empty_details(user_preferences,res)
    ask_for = check_what_is_empty(user_preferences)
    return user_preferences, ask_for


In [689]:
user_preferences = PreferenceDetails()
ask_for = check_what_is_empty(user_preferences)

# while loop until all info are collected
while ask_for:
    ai_response = ask_for_info(ask_for) + ' Answer adding what your answer is about, e.g. "4 bedrooms"'    # THIS IS CRITICAL FOR THE LLM TO IDENTIFY THE CATEGORY SINCE NO MEMORY -> NO LINK BETWEEN QUESTION AND ANSWER (answer must make the obvious link with the subject of the question)
    print(YELLOW + ai_response + ENDC)
    customer_input = input(ai_response)
    print(GREEN + customer_input + ENDC)
    user_preferences, ask_for = filter_response(customer_input, user_preferences)
else:
    print('Thank you ! Everything has been gathered, moving now to next screening phase')

# final question to collect the gist of the ideal property
print(YELLOW + 'Besides these criteria, describe the property you would love to buy ?' + ENDC)
customer_input = input('Besides these criteria, describe the property you would love to buy ?')
print(GREEN + customer_input + ENDC)

Field 'price' is empty.
Field 'bedrooms' is empty.
Field 'bathrooms' is empty.
Field 'size' is empty.
What is your budget for the property you are looking for? Answer adding what your answer is about, e.g. "4 bedrooms"
Field 'bedrooms' is empty.
Field 'bathrooms' is empty.
Field 'size' is empty.
How many bedrooms are you looking for in a property? Answer adding what your answer is about, e.g. "4 bedrooms"
Field 'bathrooms' is empty.
Field 'size' is empty.
How many bathrooms are you looking for in your ideal property? Answer adding what your answer is about, e.g. "4 bedrooms"
Field 'size' is empty.
What size of property are you looking for? Answer adding what your answer is about, e.g. "4 bedrooms"
Thank you ! Everything has been gathered, moving now to next screening phase
Besides these criteria, describe the property you would love to buy ?


In [690]:
user_preferences

PreferenceDetails(price=2500000, bedrooms=4, bathrooms=3, size=180)

In [691]:
# TRIGGER FUNCTION CALLING PARAMETERS
ai_response = chat_llm.predict_messages([HumanMessage(content=customer_input + ' Characteristics: ' + str(user_preferences.dict()))], functions=functions)
# similar to response = chat_llm.invoke("i am looking for a house with 3 bedrooms and a view on the lake?", functions=functions)
ai_response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'query_db', 'arguments': '{"query":"quiet neighborhood, proximity to city","n_results":3}'}})

In [692]:
if ai_response.content=='':
    # EXECUTE FUNCTION CALLING
    tool_name = ai_response.additional_kwargs['function_call']['name']
    tool = [tools[i] for i, tool in enumerate(tools) if tool.name==tool_name][0]
    _args = json.loads(ai_response.additional_kwargs['function_call']['arguments'])

    print(tool.name)
    print(_args)
    extrated_properties = tool(_args)
    print(extrated_properties)
    # narrow down the list using threshold parameter
    matches_df = display_matches(extrated_properties,0.37)
    display(matches_df)

    # generate augmented personalized descriptions
    augmented_df = augment_properties(extrated_properties, customer_input, matches_df)
    # Generate final LLM response in natural language
    final_output = chat_llm.predict_messages([ 
            HumanMessage(content=customer_input),
            ai_response,
            FunctionMessage(name=tool.name, content=str(extrated_properties)),
            AIMessage(content=str((augmented_df
                                .drop(['listings_description','listings_neighborhood_description','full_description'], axis=1,errors='ignore')  # we keep the augmented description and the metadata characteristics
                                .to_dict(orient='dict')
                                )
                                ))
                                ], 
                                functions=functions
                            )
    # Print final recommendation in natural language
    display(Markdown(final_output.content))

query_db
{'query': 'quiet neighborhood, proximity to city', 'n_results': 3}
{'ids': [['5', '14', '15']], 'distances': [[0.3490514690109264, 0.36467161012339894, 0.36591028592542746]], 'metadatas': [[{'bathrooms': 2, 'bedrooms': 3, 'house size': 180, 'neighborhood': 'Downtown District', 'price': 2000000}, {'bathrooms': 2, 'bedrooms': 3, 'house size': 170, 'neighborhood': 'Cedar Valley', 'price': 1600000}, {'bathrooms': 3, 'bedrooms': 4, 'house size': 260, 'neighborhood': 'City Center', 'price': 2500000}]], 'embeddings': None, 'documents': [['Urban Chic in Downtown District: Embrace urban chic living in this stylish 3-bedroom, 2-bathroom home in the heart of the Downtown District. With modern design, high-end appliances, and a rooftop patio for entertaining, this property is perfect for those who love city living. Neighborhood: The Downtown District is a bustling urban center with a vibrant nightlife, trendy restaurants, and cultural attractions. Experience the excitement of city life ri

,listings_headline,listings_neighborhood,listings_price,listings_bedrooms,listings_bathrooms,listings_house_size,listings_description,listings_neighborhood_description,full_description
5,Urban Chic in Downtown District,Downtown District,2000000,3,2,180,Embrace urban chic living in this stylish 3-be...,The Downtown District is a bustling urban cent...,Urban Chic in Downtown District: Embrace urban...
14,Tranquil Retreat in Cedar Valley,Cedar Valley,1600000,3,2,170,"Find peace and tranquility in this 3-bedroom, ...",Cedar Valley is a peaceful neighborhood with h...,Tranquil Retreat in Cedar Valley: Find peace a...
15,Urban Oasis in City Center,City Center,2500000,4,3,260,Discover an urban oasis in the heart of the Ci...,City Center is a vibrant urban hub with shoppi...,Urban Oasis in City Center: Discover an urban ...


Distance 0.349 : Urban Chic in Downtown District
- neighborhood              : Downtown District
- price                     : 2000000
- bedrooms                  : 3
- bathrooms                 : 2
- house size                : 180


<span style="color:cyan">augmented description : 
Looking for the perfect blend of city living and peaceful surroundings? Look no further than this stunning 3-bedroom, 2-bathroom home in the heart of the Downtown District. With its modern design, high-end appliances, and rooftop patio for entertaining, this property is the epitome of urban chic. And with its location in a quiet neighborhood just steps away from the bustling city center, you can have the best of both worlds. Don't miss out on the opportunity to experience the excitement of city life while still enjoying a tranquil living environment.</span>

-------------------------------------------------
Distance 0.365 : Tranquil Retreat in Cedar Valley
- neighborhood              : Cedar Valley
- price                     : 1600000
- bedrooms                  : 3
- bathrooms                 : 2
- house size                : 170


<span style="color:cyan">augmented description : 
Welcome to your tranquil retreat in Cedar Valley! This stunning 3-bedroom, 2-bathroom home is nestled among the beautiful cedar trees, providing a peaceful and serene atmosphere. Step onto the wrap-around porch and take in the natural surroundings, or find your inner peace in the meditation garden. Located in the quiet neighborhood of Cedar Valley, you'll have access to hiking trails, nature reserves, and a strong sense of community. Enjoy the best of both worlds with a peaceful escape from the city, while still being in close proximity. Don't miss out on this idyllic enclave - schedule a viewing today!</span>

-------------------------------------------------
Distance 0.366 : Urban Oasis in City Center
- neighborhood              : City Center
- price                     : 2500000
- bedrooms                  : 4
- bathrooms                 : 3
- house size                : 260


<span style="color:cyan">augmented description : 
Welcome to your dream home in the City Center! This stunning 4-bedroom, 3-bathroom oasis offers the perfect balance of tranquility and city living. With a modern design and a rooftop terrace boasting breathtaking city views, you'll never want to leave. Need to work from home? No problem, this home also features a convenient home office. And with its prime location in the bustling City Center, you'll have easy access to shopping, dining, and cultural attractions. Say goodbye to noisy neighborhoods and hello to your peaceful urban retreat.</span>

-------------------------------------------------


Based on your preferences, here are the top 3 property matches:

1. Urban Chic in Downtown District
   - Neighborhood: Downtown District
   - Price: $2,000,000
   - Bedrooms: 3
   - Bathrooms: 2
   - House Size: 180 sq. ft
   - Description: Looking for the perfect blend of city living and peaceful surroundings? Look no further than this stunning 3-bedroom, 2-bathroom home in the heart of the Downtown District. With its modern design, high-end appliances, and rooftop patio for entertaining, this property is the epitome of urban chic. And with its location in a quiet neighborhood just steps away from the bustling city center, you can have the best of both worlds. Don't miss out on the opportunity to experience the excitement of city life while still enjoying a tranquil living environment.

2. Tranquil Retreat in Cedar Valley
   - Neighborhood: Cedar Valley
   - Price: $1,600,000
   - Bedrooms: 3
   - Bathrooms: 2
   - House Size: 170 sq. ft
   - Description: Welcome to your tranquil retreat in Cedar Valley! This stunning 3-bedroom, 2-bathroom home is nestled among the beautiful cedar trees, providing a peaceful and serene atmosphere. Step onto the wrap-around porch and take in the natural surroundings, or find your inner peace in the meditation garden. Located in the quiet neighborhood of Cedar Valley, you'll have access to hiking trails, nature reserves, and a strong sense of community. Enjoy the best of both worlds with a peaceful escape from the city, while still being in close proximity. Don't miss out on this idyllic enclave - schedule a viewing today!

3. Urban Oasis in City Center
   - Neighborhood: City Center
   - Price: $2,500,000
   - Bedrooms: 4
   - Bathrooms: 3
   - House Size: 260 sq. ft
   - Description: Welcome to your dream home in the City Center! This stunning 4-bedroom, 3-bathroom oasis offers the perfect balance of tranquility and city living. With a modern design and a rooftop terrace boasting breathtaking city views, you'll never want to leave. Need to work from home? No problem, this home also features a convenient home office. And with its prime location in the bustling City Center, you'll have easy access to shopping, dining, and cultural attractions. Say goodbye to noisy neighborhoods and hello to your peaceful urban retreat.

# OUT

In [511]:
'''system_template="You are a real estate agent. You engage conversation with the customers to understand their needs and provide them with the best possible property. Your task is to collect the necessary selection criteria of the customer."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
ai_template_1 = "Hi! I am here to help you find the best property for you. Could you please tell me about the ideal characteristics of the property you are lookiging for?"
ai_message_1= AIMessagePromptTemplate.from_template(ai_template_1)
human_template="{customer_input}" 
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
ai_template_2="{response}"
ai_message_2 = AIMessagePromptTemplate.from_template(ai_template_2)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,ai_message_1, human_message_prompt,ai_message_2])
llm_chain = LLMChain(prompt=chat_prompt, llm=chat_llm)
customer_input="i am looking for a house with 3 bedrooms and a view on the lake. What do you propose?"
response1 = llm_chain.run(response="", customer_input=customer_input, functions=functions)
display(Markdown(response1))'''

Great! A house with 3 bedrooms and a view of the lake sounds wonderful. Are there any other specific features or amenities you are looking for in the property? For example, do you prefer a modern or traditional style, a certain neighborhood, or any specific budget in mind?

In [ ]:

"""

prompt = '''You are a helpful telcom sales assistant. In order to help someone to the right plan you need the city a person is based on wether the person is look for an individual plan or a family plan. 

For an individual plan- just ask the age of the person. 

If It is a family plan - we need to know how many people are on it and what there ages are for each person. 
The max for a family plan is set to 5. 
In order to provide a quote - you will need to have the ages of all people on the plan.

When people provide only the city name, please infer the State and then confirm with the person. Like when they say 'Atlanta' - you ask something like 'So you are in Atlanta, Georgia right?

Once you have all the information (Individual or family plan, number of people and age for EACH person, city and state) you can call the plan function that will return the price. '''
functions = [

{
  "name": "getplanprice",
  "parameters": {
    "type": "object",
    "properties": {
      "plan": {
        "type": "string",
        "Description": "Individual or Family"
      },
      "people": {
        "type": "integer",
        "description": "The number of people on the plan"
      },
      "ages": {
        "type": "string",
        "description": "Commma separates list of the ages of all plan participants"
      }
    },
    "required": [
      "plan",
      "people",
      "ages"
    ]
  },
  "description": "Needs the city/state, the type of plan, the number of peole on the plan and the ages of the people"
}


]

- we use a set of defined Q & A
personal_questions = [  "Which movie genre you like the most?", 
                        "What is your favorite color?", 
                        "What is your favorite movie?", 
                        "Pick one - dogs, cats or hamsters?",
                        "What is your favorite food?",
                        "What is your favorite drink?" ]

#personal_answers = [ ] 
#for question in personal_questions:
#    answer = input(question)
#    personal_answers.append(answer)
    
personal_answers = ['thriller', 'blue', 'inception', 'dogs', 'fish tacos', 'cold beer']"""